# load dask cluster with data
load a parquet dataset into a dask cluster

In [ ]:
# nuclio: ignore
import nuclio

In [ ]:
%nuclio config kind = "dask"
%nuclio config spec.image = "mlrun/ml-models"

In [ ]:
import os
import json
import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import List, Optional

def load_dask(
    context: MLClientCtx,
    src_data: DataItem,
    dask_key: str = "dask_key",
    inc_cols: List[str] = [],
    index_cols: List[str] = [],
    dask_persist: bool = True,
    refresh_data: bool = True,
    scheduler_key: str = "scheduler"
) -> None:
    """Load dataset into an existing dask cluster
    
    dask jobs define the dask client parameters at the job level, this method will raise an error if no client is detected.
    
    :param context:         the function context
    :param src_data:        url of the data file or partitioned dataset as either
                            artifact DataItem, string, or path object (similar to 
                            pandas read_csv)
    :param dask_key:        destination key of data on dask cluster and artifact store
    :param inc_cols:        include only these columns (very fast)
    :param index_cols:      list of index column names (can be a long-running process)
    :param dask_persist:    (True) should the data be persisted (through the `client.persist` op)
    :param refresh_data:    (False) if the dask_key already exists in the dask cluster, this will 
                            raise an Exception.  Set to True to replace the existing cluster data.
    :param scheduler_key:   (scheduler) the dask scheduler configuration, json also logged as an artifact
    """
    if hasattr(context, "dask_client"):
        dask_client = context.dask_client
    else:
        raise Exception("a dask client was not found in the execution context")
    
    df = src_data.as_df(df_module=dd)

    if dask_persist:
        df = dask_client.persist(df)
        if dask_client.datasets and dask_key in dask_client.datasets:
            dask_client.unpublish_dataset(dask_key)
        dask_client.publish_dataset(df, name=dask_key)
    
    if context:
        context.dask_client = dask_client
        
    # share the scheduler, whether data is persisted or not
    dask_client.write_scheduler_file(scheduler_key+".json")
    
    # we don't use log_dataset here until it can take into account
    # dask origin and apply dask describe.
    context.log_artifact(scheduler_key, local_path=scheduler_key+".json")

In [ ]:
# nuclio: end-code

### mlconfig

In [ ]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'

### save

In [ ]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dask", handler="load_dask")

# add metadata (for templates and reuse)
fn.spec.description = "load dask cluster with data"
fn.metadata.categories = ["data-movement", "utils"]
fn.metadata.labels = {"author": "yjb"}
fn.spec.remote = True
fn.spec.replicas = 4
fn.spec.max_replicas = 8
fn.spec.service_type = "NodePort"

fn.save()
fn.export("function.yaml")

### test

In [ ]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [ ]:
from mlrun import NewTask, run_local

task_params = {
    "name":        "tasks load dask cluster with data",
    "params" : {
        "persist"      : True,
        "refresh_data" : True,
        "dask_key"     : "dask_key"}}

In [ ]:
run = fn.run(NewTask(**task_params), 
               handler=load_dask, 
               inputs={"src_data" : os.path.join(mlconf.artifact_path, 'iris.parquet') },
               artifact_path=mlconf.artifact_path)

In [ ]:
fn.status.to_dict()

In [ ]:
import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

### TODO: this client dash board wont work -- wrong port!

...even though its the correct client

In [ ]:
client = Client(func.status.to_dict()['scheduler_address'])
client

In [ ]:
list(client.list_datasets())

In [ ]:
client.datasets['dask_key']